In [1]:
import numpy as np 
import pandas as pd
import os
import joblib
import sklearn 
import matplotlib
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

#Regressions:

from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#Metric
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


from pandas import DataFrame


# Show progress bar
from tqdm import tqdm

In [2]:
df = pd.read_csv('hao_augmented_dataset.csv')
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,in_amount_mmol,p_amount_mmol,sol_amount_ml,TOP_amount_mmol,acid_amount_mmol,amine_amount_mmol,...,x8_oleic acid,x8_superhydride,x8_tetrabutylammonium myristate,x8_trioctylamine,x8_trioctylphosphine oxide,x8_water,x8_zinc iodide,diameter_nm,abs_nm,emission_nm
0,0,0,0,0,2.798293,0.769563,-0.528015,1.951881,-0.431077,-0.641827,...,0,0,0,0,1,0,0,2.753333,599.0,612.000000
1,1,1,1,1,0.055436,-0.361432,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,0,1,0,0,2.600000,530.0,613.000000
2,2,2,2,2,0.782458,0.017999,-0.528015,0.600491,-0.431077,-0.641827,...,0,0,0,0,1,0,0,3.500000,595.0,610.000000
3,3,4,4,4,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,0,1,0,0,2.520000,520.0,612.000000
4,4,5,5,5,0.055436,0.280681,-0.528015,1.958217,-0.431077,-0.641827,...,0,0,0,0,1,0,0,2.660000,590.0,612.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,212,214,214,214,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,0,5.433333,570.0,605.000000
213,213,215,215,215,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,0,5.433333,562.0,605.000000
214,214,216,216,216,-0.580709,-0.667895,-0.070987,-0.304661,-0.316033,-0.641827,...,0,0,0,0,0,0,0,5.433333,549.0,589.666667
215,215,217,217,217,-0.423738,-0.631411,-0.201566,-0.304661,-0.085945,-0.600008,...,0,0,0,0,0,0,0,2.000000,523.0,597.000000


In [3]:
# Input for ML models

input_col = ['in_amount_mmol',
             'p_amount_mmol',
             'sol_amount_ml',
             'TOP_amount_mmol',
             'acid_amount_mmol',
             'amine_amount_mmol',
             'thiol_amount_mmol',
             'zinc_amount_mmol',
             'other_amount_mmol',
             'total_volume_ml',
             'temp_c',
             'time_min',
             'x0_chloroindium oxalate',
             'x0_indium acetate',
             'x0_indium bromide',
             'x0_indium chloride',
             'x0_indium iodide',
             'x0_indium myristate',
             'x0_indium oxalate',
             'x0_indium palmitate',
             'x0_indium trifluoroacetate',
             'x0_indium tris(N,N-diisopropylacetamidinato)',
             'x1_bis(trimethylsilyl)phosphine',
             'x1_phosphine gas',
             'x1_phosphorus trichloride',
             'x1_sodium phosphide',
             'x1_tris(diethylamino)phosphine',
             'x1_tris(dimethylamino)phosphine',
             'x1_tris(trimethylgermyl)phosphine',
             'x1_tris(trimethylsilyl)phosphine',
             'x1_white phosphorus',
             'x2_4-ethylpyridine',
             'x2_None',
             'x2_dimethylformamide',
             'x2_mesitylene',
             'x2_octadecene',
             'x2_toluene',
             'x3_None',
             'x3_trioctylphosphine',
             'x4_None',
             'x4_lauric acid',
             'x4_myristic acid',
             'x4_oleic acid',
             'x4_palmitic acid',
             'x4_stearic acid',
             'x5_None',
             'x5_dioctylamine',
             'x5_dodecylamine',
             'x5_hexadecylamine',
             'x5_octylamine',
             'x5_oleylamine',
             'x6_None',
             'x6_dodecanethiol',
             'x7_None',
             'x7_zinc acetate',
             'x7_zinc bromide',
             'x7_zinc chloride',
             'x7_zinc iodide',
             'x7_zinc octanoate',
             'x7_zinc oleate',
             'x7_zinc stearate',
             'x7_zinc undecylenate',
             'x8_None',
             'x8_acetic acid',
             'x8_copper bromide',
             'x8_superhydride',
             'x8_oleic acid',
             'x8_tetrabutylammonium myristate',
             'x8_trioctylamine',
             'x8_trioctylphosphine oxide',
             'x8_water',
             'x8_zinc iodide'
            ]

output_col = ['diameter_nm', 'abs_nm', 'emission_nm']

X = df[input_col]

Y = df[output_col]

In [4]:
# Splitting dataset for training

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=45, shuffle=True)

In [5]:
Y.shape

(217, 3)

In [6]:
X.shape

(217, 72)

In [7]:
#This is used to roughly show which regression performs better.
# Testing Regressions:

REGRESSIONS = {
    "Extra trees": ExtraTreesRegressor(n_estimators=10,
                                       max_features=32,     
                                       random_state=44),
    "K-nn": KNeighborsRegressor(),                          
    "Linear regression": LinearRegression(),
    "Ridge": RidgeCV(),
    "Lasso": Lasso(),
    "ElasticNet": ElasticNet(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
    "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
    "MultiO/P GBR" :MultiOutputRegressor(GradientBoostingRegressor(n_estimators=5)),
    "MultiO/P AdaB" :MultiOutputRegressor(AdaBoostRegressor(n_estimators=5))
            }
# r2 is used to evaluate the performance of all regressions.

r2_list = list()

for name, reg in REGRESSIONS.items():     
    reg.fit(X_train, Y_train)                 
    Y_pred = pd.DataFrame(reg.predict(X_test))
    
    print(name, '\n')
    
# This loop will show r2 for each outcome
    for column in range(0, 3):
        r2 = r2_score(Y_test.iloc[:, column], Y_pred.iloc[:, column])
        r2_list.append(r2)
    
    print(' R^2 for diameter is ', r2_list[0], '\n',
          'R^2 for Absorbance is ', r2_list[1], '\n',
          'R^2 for PL is ', r2_list[2], '\n', '\n', 
         )
    del r2_list[:] #reset the list for the next regression

Extra trees 

 R^2 for diameter is  0.4266524581551444 
 R^2 for Absorbance is  0.5589248933210326 
 R^2 for PL is  0.6751512082893167 
 

K-nn 

 R^2 for diameter is  0.4834701677147135 
 R^2 for Absorbance is  0.41991604439598174 
 R^2 for PL is  0.44462398193952946 
 

Linear regression 

 R^2 for diameter is  -2.133624409139695e+24 
 R^2 for Absorbance is  -7.607633583429463e+24 
 R^2 for PL is  -9.833774627654037e+24 
 

Ridge 

 R^2 for diameter is  0.268564878207722 
 R^2 for Absorbance is  0.34316505731688096 
 R^2 for PL is  0.6763196035182273 
 

Lasso 

 R^2 for diameter is  -0.015778929527688446 
 R^2 for Absorbance is  0.36336722127224097 
 R^2 for PL is  0.46432507875658613 
 

ElasticNet 

 R^2 for diameter is  -0.015778929527688446 
 R^2 for Absorbance is  0.25326140676299247 
 R^2 for PL is  0.34904074078860325 
 

RandomForestRegressor 

 R^2 for diameter is  0.2368845421565574 
 R^2 for Absorbance is  0.43432683832863306 
 R^2 for PL is  0.5797637072731807 
 

Decisi

## Optimizing


### 1. Extra Trees

In [8]:
# This is a grid search for three parameters in the Extra Trees algorithm. 
# Parameters are: random_state, n_estimators, max_features.

# This gives the best combination of the three parameters for the smallest mean squared error.

min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 25)):
    for j in range(1, 25):
        for k in range(2, 50, 1):
            ET_regr = ExtraTreesRegressor(n_estimators=i, 
                                            max_features=j,
                                            random_state=k)
            
            ET_regr.fit(X_train, Y_train)
            ET_Y_pred = pd.DataFrame(ET_regr.predict(X_test))

            mae = mean_absolute_error(Y_test, ET_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 24/24 [08:03<00:00, 20.15s/it]

10.879511782964649 2 20 23


In [9]:

ET_regr = ExtraTreesRegressor(n_estimators=2,
                            max_features=20,     
                            random_state=23).fit(X_train, Y_train)
            
ET_regr.fit(X_train, Y_train)            
ET_Y_pred = ET_regr.predict(X_test)

outputs = ('diameter: ', 'absorbance: ', 'emission: ')

for i in range(0, 3):
    ET_r2 = r2_score(Y_test.iloc[:, i], pd.DataFrame(ET_Y_pred).loc[:, i])
    ET_MSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(ET_Y_pred).loc[:, i])
    ET_RMSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(ET_Y_pred).loc[:, i], squared=False)
    ET_MAE = mean_absolute_error(Y_test.iloc[:, i], pd.DataFrame(ET_Y_pred).loc[:, i])
    print(outputs[i], 'r2:', ET_r2, '; MSE:', ET_MSE, '; RMSE:', ET_RMSE, '; MAE:', ET_MAE )

diameter:  r2: 0.5226717146087895 ; MSE: 0.5153475587545118 ; RMSE: 0.7178771195368409 ; MAE: 0.4718686867727273
absorbance:  r2: 0.7241021737535902 ; MSE: 1002.882996601852 ; RMSE: 31.668327972942492 ; MAE: 20.28787878636364
emission:  r2: 0.830417511744725 ; MSE: 425.146464487374 ; RMSE: 20.619080107690884 ; MAE: 11.878787875757581


### 2. Decision Tree 

In [10]:
# This is a grid search for three parameters in the Decision Trees algorithm. 
# Parameters are: max_depth, max_features, random_state.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mae = 99999

min_i, min_j, min_k  = 0, 0, 0

for i in tqdm(range(1, 21)):
    for j in range(1, 21):
        for k in range(5, 80, 5):
            
            DT_regr = DecisionTreeRegressor(max_depth=i,
                                max_features=j,
                                random_state=k)
            
            DT_regr.fit(X_train, Y_train)
            DT_Y_pred = pd.DataFrame(DT_regr.predict(X_test))

            mae = mean_absolute_error(Y_test, DT_Y_pred)
            
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 20/20 [00:36<00:00,  1.84s/it]

12.373406282977276 17 15 20


In [11]:
DT_regr = DecisionTreeRegressor(max_depth=13,
                                max_features=9,
                                random_state=10)
            
DT_regr.fit(X_train, Y_train)            
DT_Y_pred = DT_regr.predict(X_test)

outputs = ('diameter: ', 'absorbance: ', 'emission: ')

for i in range(0, 3):
    DT_r2 = r2_score(Y_test.iloc[:, i], pd.DataFrame(DT_Y_pred).loc[:, i])
    DT_MSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(DT_Y_pred).loc[:, i])
    DT_RMSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(DT_Y_pred).loc[:, i], squared=False)
    DT_MAE = mean_absolute_error(Y_test.iloc[:, i], pd.DataFrame(DT_Y_pred).loc[:, i])
    print(outputs[i], 'r2:', DT_r2, '; MSE:', DT_MSE, '; RMSE:', DT_RMSE, '; MAE:', DT_MAE )

diameter:  r2: 0.09241622102366664 ; MSE: 0.9798729703966971 ; RMSE: 0.9898853319434009 ; MAE: 0.6765219445658338
absorbance:  r2: 0.1477935710543501 ; MSE: 3097.7530660972293 ; RMSE: 55.65746190850989 ; MAE: 41.633629528534726
emission:  r2: 0.4902467614551588 ; MSE: 1277.9608871058474 ; RMSE: 35.74857881239263 ; MAE: 24.710830107461845


### 3. Random Forest

In [13]:
# This is a grid search for three parameters in the Random Forest algorithm. 
# Parameters are: max_depth, n_estimators, max_features.
# Random_state is set to 45.
# This gives the best combination of the three parameters for the smallest mean squared error.

min_mae = 99999
min_i, min_j, min_k = 0, 0, 0
for i in tqdm(range(1, 31)):
    for j in range(1, 31):
        for k in range(2, 50, 1):
            RF_regr = RandomForestRegressor(max_depth=i, 
                                            n_estimators=j, 
                                            max_features=k,
                                            random_state=45)
            RF_regr.fit(X_train, Y_train)
            RF_Y_pred = pd.DataFrame(RF_regr.predict(X_test))

            mae = mean_absolute_error(Y_test, RF_Y_pred)
            if (min_mae > mae):
                min_mae = mae
                min_i = i
                min_j = j
                min_k = k
            
print(min_mae, min_i, min_j, min_k)

100%|██████████| 30/30 [17:29<00:00, 34.99s/it]

13.498290956239023 14 8 6


In [37]:
RF_regr = RandomForestRegressor(max_depth=15, 
                                n_estimators=6, 
                                max_features=45,
                                random_state=45)
            
RF_regr.fit(X_train, Y_train)            
RF_Y_pred = RF_regr.predict(X_test)

outputs = ('diameter: ', 'absorbance: ', 'emission: ')

for i in range(0, 3):
    RF_r2 = r2_score(Y_test.iloc[:, i], pd.DataFrame(RF_Y_pred).loc[:, i])
    RF_MSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(RF_Y_pred).loc[:, i])
    RF_RMSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(RF_Y_pred).loc[:, i], squared=False)
    RF_MAE = mean_absolute_error(Y_test.iloc[:, i], pd.DataFrame(RF_Y_pred).loc[:, i])
    print(outputs[i], 'r2:', RF_r2, '; MSE:', RF_MSE, '; RMSE:', RF_RMSE, '; MAE:', RF_MAE )

diameter:  r2: 0.33764141010425563 ; MSE: 1.1115826191109626 ; RMSE: 1.0543161855491752 ; MAE: 0.7323104443390259
absorbance:  r2: 0.6815129446618853 ; MSE: 1380.4011089991168 ; RMSE: 37.15374959541926 ; MAE: 23.32659633745971
emission:  r2: 0.6270426982174335 ; MSE: 1224.9707283873392 ; RMSE: 34.999581831606775 ; MAE: 23.733777597060328


### 4. K Neighbors

In [40]:
min_mae = 99999
min_i, min_j = 0, 0

for i in tqdm(range(1, 30)):
    for j in range(1, 20):

        KNN_reg = KNeighborsRegressor(n_neighbors=i, 
                                      p=j).fit(X_train, Y_train)

        KNN_Y_pred = KNN_reg.predict(X_test)

        KNN_r2 = r2_score(Y_test, pd.DataFrame(KNN_Y_pred))

        if (min_mae > mae):
            min_mae = mae
            min_i = i
            min_j = j

print(min_mae, min_i, min_j)

100%|██████████| 29/29 [00:09<00:00,  2.97it/s]

18.478823529411766 1 1


In [52]:
KNN_reg = KNeighborsRegressor(n_neighbors=1, 
                              p=1).fit(X_train, Y_train)

KNN_Y_pred = KNN_reg.predict(X_test)

outputs = ('diameter:', 'Abs:', 'PL:')
for i in range(0, 3):
    KNN_r2 = r2_score(Y_test.iloc[:, i], pd.DataFrame(KNN_Y_pred).loc[:, i])
    KNN_MSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(KNN_Y_pred).loc[:, i])
    KNN_RMSE = mean_squared_error(Y_test.iloc[:, i], pd.DataFrame(KNN_Y_pred).loc[:, i], squared=False)
    KNN_MAE = mean_absolute_error(Y_test.iloc[:, i], pd.DataFrame(KNN_Y_pred).loc[:, i])
    print(outputs[i], 'r2:', KNN_r2, '; MSE:', KNN_MSE, '; RMSE:', KNN_RMSE, '; MAE:', KNN_MAE)

diameter: r2: 0.3864332337862958 ; MSE: 1.0296992647058822 ; RMSE: 1.0147409840475954 ; MAE: 0.7133823529411765
Abs: r2: 0.6174399016788297 ; MSE: 1658.109411764706 ; RMSE: 40.719889633503506 ; MAE: 28.7
PL: r2: 0.641976769263004 ; MSE: 1175.9200735294119 ; RMSE: 34.29169102755669 ; MAE: 19.38088235294118


### 5. Lasso

In [54]:
min_mae = 9999

min_i, min_j = 0, 0

for i in tqdm(np.arange(0.1, 2.0, 0.02)):
    for j in range(1, 100):
        L_reg = Lasso(alpha=i, random_state=j).fit(X_train, Y_train)
        
        L_Y_pred = L_reg.predict(X_test)
        L_r2 = r2_score(Y_test, pd.DataFrame(L_Y_pred))
      
        if (min_mae > mae):
            min_mae = mae
            min_i = i
            min_j = j

            
print(min_mae, min_i, min_j)

  0%|          | 0/95 [00:00<?, ?it/s]/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 593.291900984419, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/lin

...

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 118.9219423076429, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/lin

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/

/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 134.49652915023034, tolerance: 49.637031546542566
  model = cd_fast.enet_coordinate_descent(
/home/anhhaokudo/miniconda3/lib/python3.8/site-packages/sklearn/

18.478823529411766 0.1 1


### 6. Neural network model - multilayer perceptron

In [72]:
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense

 
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.compile(loss='mae', optimizer='adam')
    return model
 
# # load dataset
# X, y = get_dataset()
# n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(X, Y)
# fit the model on all data
model.fit(X, Y, verbose=0, epochs=100)
# # make a prediction for new data
# row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
# newX = asarray([row])
# yhat = model.predict(newX)
# print('Predicted: %s' % yhat[0])

TypeError: Dimension value must be integer or None or have an __index__ method, got value '     in_amount_mmol  p_amount_mmol  sol_amount_ml  TOP_amount_mmol  \
0          2.786003       0.780482      -0.530171         1.968116   
1          0.042197      -0.356142      -0.530171         1.974491   
2          0.769471       0.025177      -0.530171         0.608353   
3          0.769471       0.025177      -0.530171         0.608353   
4          0.042197       0.289167      -0.530171         1.974491   
..              ...            ...            ...              ...   
217       -0.594167      -0.664131      -0.070056        -0.302405   
218       -0.594167      -0.664131      -0.070056        -0.302405   
219       -0.594167      -0.664131      -0.070056        -0.302405   
220       -0.437142      -0.627466      -0.201518        -0.302405   
221       -0.437142      -0.627466      -0.201518        -0.302405   

     acid_amount_mmol  amine_amount_mmol  thiol_amount_mmol  zinc_am

### 7.